In [194]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import (
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta

from scipy.stats import norm
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [195]:
df = pd.read_csv("data/cowichan_historic.csv")
df

,date,species,count
0,2014-05-01,CN,7
1,2014-05-02,CN,34
2,2014-05-07,CN,21
3,2014-05-08,CN,136
4,2014-05-13,CN,74
...,...,...,...
482,2020-11-20,CO,4
483,2020-11-27,CO,4
484,2020-11-28,CO,2
485,2020-11-29,CO,1


In [196]:
temp_df = pd.read_csv("data/northcochiwan_daily_temp-2.csv")
temp_df

,UTC_DATE,RELATIVE_HUMIDITY,WIND_SPEED,TEMP,WINDCHILL,DEW_POINT_TEMP
0,2013-09-02,75.818182,2.727273,19.127273,NaN,14.372727
1,2013-09-03,83.125000,2.458333,18.045833,NaN,14.766667
2,2013-09-04,85.791667,2.000000,17.062500,NaN,14.379167
3,2013-09-05,94.708333,1.541667,16.837500,NaN,15.900000
4,2013-09-06,91.916667,1.583333,16.954167,NaN,15.504167
...,...,...,...,...,...,...
3918,2024-05-26,75.125000,6.791667,11.083333,NaN,6.545833
3919,2024-05-27,79.166667,4.375000,12.491667,NaN,8.745833
3920,2024-05-28,78.708333,3.291667,12.520833,NaN,8.616667
3921,2024-05-29,67.625000,4.791667,11.154167,NaN,4.970833


In [197]:
comb = df.merge(temp_df[["UTC_DATE", "TEMP"]], left_on="date", right_on="UTC_DATE", how="right")
comb = comb.drop(["date", "species"], axis=1)
comb["count"] = comb["count"].fillna(0)
comb = comb.rename(columns={"UTC_DATE": "date"})
comb["date"] = pd.to_datetime(comb["date"])
comb["month"] = comb["date"].dt.month
comb["year"] = comb["date"].dt.year
comb

,count,date,TEMP,month,year
0,0.0,2013-09-02,19.127273,9,2013
1,0.0,2013-09-03,18.045833,9,2013
2,0.0,2013-09-04,17.062500,9,2013
3,0.0,2013-09-05,16.837500,9,2013
4,0.0,2013-09-06,16.954167,9,2013
...,...,...,...,...,...
3980,0.0,2024-05-26,11.083333,5,2024
3981,0.0,2024-05-27,12.491667,5,2024
3982,0.0,2024-05-28,12.520833,5,2024
3983,0.0,2024-05-29,11.154167,5,2024


In [198]:
comb["month"].unique()

array([ 9, 10, 11, 12,  1,  2,  3,  4,  5,  6,  7,  8], dtype=int32)

In [199]:
def parse_winter(df):
      winter_months = [10, 11, 12,1,2]

      winter_df = df[df["month"].isin(winter_months)]
      df["december_temp"] = 0
      df["january_temp"] = 0
      df["feburary_temp"] = 0
      df["october_temp"] = 0
      df["november_temp"] = 0

      for year in df["year"].unique().tolist():
            temp_df = df[df["year"] == year-1]

            for month in winter_months:
                  month_df = temp_df[temp_df["month"] == month]
                  month_avg = month_df["TEMP"].mean()

                  if month == 12:
                        df.loc[(df["year"] == year), "december_temp"] = month_avg
                  elif month == 10:
                        df.loc[(df["year"] == year), "october_temp"] = month_avg
                  elif month == 11:
                        df.loc[(df["year"] == year), "november_temp"] = month_avg
                  elif month == 1:
                        df.loc[(df["year"] == year), "january_temp"] = month_avg
                  elif month == 2:
                        df.loc[(df["year"] == year), "feburary_temp"] = month_avg

      return df

In [200]:
comb_ = parse_winter(comb)
comb_ = comb_[(comb_["year"] > 2015) & (comb_["year"] <= 2020)]
comb_.tail(5)

,count,date,TEMP,month,year,december_temp,january_temp,feburary_temp,october_temp,november_temp
2734,0.0,2020-12-27,3.604545,12,2020,4.676562,4.324905,0.100744,7.912634,5.528056
2735,0.0,2020-12-28,5.133333,12,2020,4.676562,4.324905,0.100744,7.912634,5.528056
2736,0.0,2020-12-29,2.141667,12,2020,4.676562,4.324905,0.100744,7.912634,5.528056
2737,0.0,2020-12-30,4.216667,12,2020,4.676562,4.324905,0.100744,7.912634,5.528056
2738,0.0,2020-12-31,6.418182,12,2020,4.676562,4.324905,0.100744,7.912634,5.528056


In [201]:
def rolling_temp(df):
      df["rolling_mean"] = df["TEMP"].rolling(30).mean()
      df["rolling_std"] = df["TEMP"].rolling(30).std()
      return df

def lag_df(df, numeric_cols, lag=1):
    lagged_df = pd.DataFrame()
    for col in numeric_cols:
        for i in range(lag + 1):
            lagged_df[f'{col}_t-{i}'] = df[col].shift(i)
    return lagged_df.dropna()

comb_ = rolling_temp(comb_)
# comb_lag = lag_df(comb_, ["TEMP"], lag=10)
# comb_ = pd.concat([comb_, comb_lag], axis=1)
comb_

/var/folders/c5/pphx1c0n3lb12sy85_ybqxqm0000gn/T/ipykernel_13250/1580726659.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rolling_mean"] = df["TEMP"].rolling(30).mean()
/var/folders/c5/pphx1c0n3lb12sy85_ybqxqm0000gn/T/ipykernel_13250/1580726659.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rolling_std"] = df["TEMP"].rolling(30).std()


,count,date,TEMP,month,year,december_temp,january_temp,feburary_temp,october_temp,november_temp,rolling_mean,rolling_std
850,0.0,2016-01-01,-3.008333,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,NaN,NaN
851,0.0,2016-01-02,-2.387500,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,NaN,NaN
852,0.0,2016-01-03,-1.479167,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,NaN,NaN
853,0.0,2016-01-04,1.016667,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,NaN,NaN
854,0.0,2016-01-05,-0.091667,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2734,0.0,2020-12-27,3.604545,12,2020,4.676562,4.324905,0.100744,7.912634,5.528056,4.125538,2.670628
2735,0.0,2020-12-28,5.133333,12,2020,4.676562,4.324905,0.100744,7.912634,5.528056,4.074844,2.635173
2736,0.0,2020-12-29,2.141667,12,2020,4.676562,4.324905,0.100744,7.912634,5.528056,4.041649,2.653595
2737,0.0,2020-12-30,4.216667,12,2020,4.676562,4.324905,0.100744,7.912634,5.528056,3.954288,2.600997


In [202]:
comb_.groupby(["year"]).size()

year
2016    366
2017    368
2018    397
2019    383
2020    375
dtype: int64

In [203]:
comb_

,count,date,TEMP,month,year,december_temp,january_temp,feburary_temp,october_temp,november_temp,rolling_mean,rolling_std
850,0.0,2016-01-01,-3.008333,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,NaN,NaN
851,0.0,2016-01-02,-2.387500,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,NaN,NaN
852,0.0,2016-01-03,-1.479167,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,NaN,NaN
853,0.0,2016-01-04,1.016667,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,NaN,NaN
854,0.0,2016-01-05,-0.091667,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2734,0.0,2020-12-27,3.604545,12,2020,4.676562,4.324905,0.100744,7.912634,5.528056,4.125538,2.670628
2735,0.0,2020-12-28,5.133333,12,2020,4.676562,4.324905,0.100744,7.912634,5.528056,4.074844,2.635173
2736,0.0,2020-12-29,2.141667,12,2020,4.676562,4.324905,0.100744,7.912634,5.528056,4.041649,2.653595
2737,0.0,2020-12-30,4.216667,12,2020,4.676562,4.324905,0.100744,7.912634,5.528056,3.954288,2.600997


In [204]:
def impute_for_missing(df):
      missing_cols = df.columns[df.isna().any()].tolist()
      for col in missing_cols:
            median_value = df[col].median()
            df[col].fillna(median_value, inplace=True)
      return df

In [205]:
trial_df = comb_.copy()
trial_df = impute_for_missing(trial_df)
split_date = "2019-12-30"
display(trial_df.head(6))
trial_df = trial_df.set_index('date')
trial_df['count'] = trial_df['count'].fillna(0)
exog = trial_df[['TEMP', 'december_temp', 'january_temp', 'feburary_temp', 'october_temp', 'november_temp']]

train_data = trial_df[trial_df.index < split_date]
test_data = trial_df[trial_df.index >= split_date]
train_exog = exog[exog.index < split_date]
test_exog = exog[exog.index >= split_date]

print("Train Data Indices:", train_data.index)
print("Test Data Indices:", test_data.index)


/var/folders/c5/pphx1c0n3lb12sy85_ybqxqm0000gn/T/ipykernel_13250/2039789980.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_value, inplace=True)
/var/folders/c5/pphx1c0n3lb12sy85_ybqxqm0000gn/T/ipykernel_13250/2039789980.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

,count,date,TEMP,month,year,december_temp,january_temp,feburary_temp,october_temp,november_temp,rolling_mean,rolling_std
850,0.0,2016-01-01,-3.008333,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,9.959583,2.453405
851,0.0,2016-01-02,-2.387500,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,9.959583,2.453405
852,0.0,2016-01-03,-1.479167,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,9.959583,2.453405
853,0.0,2016-01-04,1.016667,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,9.959583,2.453405
854,0.0,2016-01-05,-0.091667,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,9.959583,2.453405
855,0.0,2016-01-06,2.262500,1,2016,4.015097,5.092313,7.176082,11.646505,4.361111,9.959583,2.453405


Train Data Indices: DatetimeIndex(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
               '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08',
               '2016-01-09', '2016-01-10',
               ...
               '2019-12-20', '2019-12-21', '2019-12-22', '2019-12-23',
               '2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27',
               '2019-12-28', '2019-12-29'],
              dtype='datetime64[ns]', name='date', length=1512, freq=None)
Test Data Indices: DatetimeIndex(['2019-12-30', '2019-12-31', '2020-01-01', '2020-01-02',
               '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06',
               '2020-01-07', '2020-01-08',
               ...
               '2020-12-22', '2020-12-23', '2020-12-24', '2020-12-25',
               '2020-12-26', '2020-12-27', '2020-12-28', '2020-12-29',
               '2020-12-30', '2020-12-31'],
              dtype='datetime64[ns]', name='date', length=377, freq=None)


In [206]:
test_data.index[0]

Timestamp('2019-12-30 00:00:00')

In [214]:
train_data[exog.columns]

,TEMP,december_temp,january_temp,feburary_temp,october_temp,november_temp
date,,,,,,
2016-01-01,-3.008333,4.015097,5.092313,7.176082,11.646505,4.361111
2016-01-02,-2.387500,4.015097,5.092313,7.176082,11.646505,4.361111
2016-01-03,-1.479167,4.015097,5.092313,7.176082,11.646505,4.361111
2016-01-04,1.016667,4.015097,5.092313,7.176082,11.646505,4.361111
2016-01-05,-0.091667,4.015097,5.092313,7.176082,11.646505,4.361111
...,...,...,...,...,...,...
2019-12-25,3.900000,3.965188,4.590595,3.289368,9.090104,6.798333
2019-12-26,1.175000,3.965188,4.590595,3.289368,9.090104,6.798333
2019-12-27,2.620833,3.965188,4.590595,3.289368,9.090104,6.798333


In [211]:
model = SARIMAX(train_data['count'], exog=train_data[exog.columns], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
model_fit = model.fit(disp=False, maxiter=100)

start_date = test_data.index[0]  # Example start date
if start_date not in test_data.index:
    print("Start date is not in the index of test_data.")
else:
    print("Start date is in the index of test_data.")

/Users/jennylee/miniforge3/envs/563/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/jennylee/miniforge3/envs/563/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Start date is in the index of test_data.


In [213]:
model_fit.forecast(1)

/Users/jennylee/miniforge3/envs/563/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


ValueError: Out-of-sample operations in a model with a regression component require additional exogenous values via the `exog` argument.

In [ ]:
pred = model_fit.get_prediction(start=test_data.index[0], end=test_data.index[-1], exog=test_exog, dynamic=False)
pred_ci = pred.conf_int()